In [84]:
import pandas as pd
# import matplotlib.pyplot as plt
# import numpy as np
# df = pd.read_csv('Data/mcts_trial_data.csv', sep='\t', encoding='utf-8')
df = pd.read_pickle('Data/mcts_rand_trial_data.pkl')
cols = ['Board', 'Moves','Move_Probs','Reward', 'Weights']

print(df.head)
print(df.ndim, ' dimensions')
print(df.shape, ' shape')
print(df.size, ' size')

<bound method NDFrame.head of                                                 Board  \
0   [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [...   
1   [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [...   
2   [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [...   
3   [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [...   
4   [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [...   
..                                                ...   
3   [[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], [...   
4   [[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], [...   
5   [[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], [...   
6   [[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], [...   
7   [[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0], [...   

                                                Moves  \
0   [[4, 4, 7, 5, 0, 0], [3, 4, 16, 3, 0, 1], [4, ...   
1   [[8, 9, 20, 3, 0, 1], [10, 8, 9, 3, 0, 2], [10...   
2   [[4, 1, 9, 1, 4, 2], [4, 1, 11, 1, 0, 1], [2, ...   
3   [[10, 11, 19, 3, 3, 0], [9, 11, 14, 5, 4, 3], ...   


In [81]:
from board import Board

def convertToMatrix(test_str, K):
    # slicing strings
    temp = [test_str[idx: idx + K] for idx in range(0, len(test_str), K)]
 
    # conversion to list of characters
    res = [list(ele) for ele in temp]
 
    # printing result
    return res
 
 
row = 100
print(df.iloc[row])
x = df.iloc[row]['Board']
board = Board(14)
board.show_dots = False
board.board = x
print(board)
print(df.iloc[row]['Move_Probs'])
print(df.iloc[row]['Moves'])


Board         [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [...
Moves         [[6, 7, 5, 4, 2, 3], [8, 2, 7, 1, 0, 2], [3, 7...
Move_Probs    [0.010869565217391304, 0.010869565217391304, 0...
Reward                                                        1
Weights       [43.3502663779507, 33.18342893503532, 28.63392...
Num_Sims                                                     92
Name: 28, dtype: object
   A B C D E F G H I J K L M N
 1 □ □ □ □ □ □ □ □ □ □ □ □ □ □ 
 2 □ □ □ □ □ □ □ □ □ □ □ □ □ □ 
 3 □ □ □ □ □ □ □ □ □ □ □ □ □ □ 
 4 □ □ ▣ ▣ □ □ ▣ □ □ □ □ □ □ □ 
 5 □ □ ▣ □ ▣ ▣ ▣ □ □ □ □ □ □ □ 
 6 □ ▣ ▣ □ □ ▣ □ □ □ □ □ □ □ □ 
 7 □ □ □ □ □ □ □ □ ▣ ▣ □ □ □ □ 
 8 □ □ □ □ □ □ □ ▣ ▣ □ □ □ □ □ 
 9 □ □ □ □ □ □ □ ▣ □ □ □ □ □ □ 
10 □ □ □ □ □ □ □ □ ▣ ▣ □ □ □ □ 
11 □ □ □ □ □ □ □ □ □ ▣ □ □ □ □ 
12 □ □ □ □ □ □ □ □ □ ▣ ▣ □ □ □ 
13 □ □ □ □ □ □ □ □ □ □ □ □ □ □ 
14 □ □ □ □ □ □ □ □ □ □ □ □ □ □ 

[0.010869565217391304, 0.010869565217391304, 0.010869565217391304, 0.010869565217391304, 0.010869565217391304, 0